# Optimizing the model with series of experiments

In [ ]:
import pathlib

import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

### Let´s first reload the data from the previous notebook. Then, perform train_test_split once again.

In [ ]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

In [ ]:
model_data_scaled_path = DATA_DIR / 'processed' / 'ames_model_data_scaled.pkl'

In [ ]:
data = pd.read_pickle(model_data_scaled_path)

In [ ]:
X = data.drop(columns=['SalePrice']).copy().values
y = data['SalePrice'].copy().values

In [ ]:
X.shape, y.shape

In [ ]:
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

### Experiment 1: LinearRegression with transformed values

Our first experiment is to rerun the linear regression model with the same features as before, but this time on the new data with the transformations applied on the previous notebook.


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_scaled_model = LinearRegression()

linear_scaled_model.fit(Xtrain, ytrain)

For this and the following experiments, we will also use cross_val_score instead of simply predicting on the test set. This will give us a better idea of how the model will perform on unseen data, given that we are making more realistic performance estimates, and the variance of the model evaluation will be lower.

In [ ]:
scores_linear_scaled = cross_val_score(linear_scaled_model,
                                       Xtrain,
                                       ytrain,
                                       cv=10,
                                       scoring='neg_mean_squared_error',
                                       n_jobs=-1)

scores_linear_scaled = np.sqrt(-scores_linear_scaled)
error_percent_linear = 100 * (10**scores_linear_scaled.mean() - 1)
std_percent_linear = 100 * (10**scores_linear_scaled.std() - 1)
print(f'Average error is {error_percent_linear:.2f}%')
print(f'Standard deviation of error is {std_percent_linear:.2f}%')

path = DATA_DIR / 'processed' / 'linear_scaled_model.csv'

# write the array scores_linear_scaled to a csv file

np.savetxt(path, scores_linear_scaled, delimiter=',')
        

The experiment showed no large difference in the model performance, considering the standard deviation. This means that the LinearRegression model is not sensitive to the new transformations we made on the data. We will, however, still keep the new data for the next experiments.

### Experiment 2: Lasso Regression

The Lasso Regression is similar to the Linear Regression, but it adds a regularization term to the cost function, which penalizes the model for having too many features (in this case, the L1 norm of the weights). This is useful to avoid overfitting, and also to perform feature selection, since the regularization term will make the weights of the less important features go to zero.

We can define this regularization term as:

$$
\lambda \sum_{i=1}^{n} |w_i|
$$

Where $\lambda$ is the regularization parameter, and $w_i$ is the weight of the $i$-th feature.

As $w_i$ gets closer to zero, the regularization term will also get closer to zero, and the model will be penalized less. This means that the model will try to minimize the cost function by making the weights of the less important features go to zero, and the weights of the most important features will be kept as they are.

In [ ]:
from sklearn.linear_model import Lasso

For this and the following experiments, we will use GridSearchCV to find better values for model hyperparameters. This will allow us to perform a more thorough search in the hyperparameter space, and find the best model for our data without having to manually test different values.

In [ ]:
lasso = Lasso()

params = {
    'alpha': np.logspace(-4, 0, 100),
    'max_iter': [15000, 20000, 30000],
    'tol': [0.0001, 0.001],
    'selection': ['cyclic', 'random'],
}

grid = GridSearchCV(lasso, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

Get the best parameters for Lasso Regression, from the grid search, and use them to train a new model. Then, evaluate the model using cross_val_score:

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
lasso_best = Lasso(**best_params)
lasso_best.fit(Xtrain, ytrain)

In [ ]:
scores_lasso = cross_val_score(lasso_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_lasso = np.sqrt(-scores_lasso)
error_percent_lasso = 100 * (10**scores_lasso.mean() - 1)
std_percent_lasso = 100 * (10**scores_lasso.std() - 1)
print(f'Average error is {error_percent_lasso:.2f}%')
print(f'Standard deviation of error is {std_percent_lasso:.2f}%')

path = DATA_DIR / 'processed' / 'lasso_score.csv'

np.savetxt(path, scores_lasso, delimiter=',')

### Experiment 3: Ridge

The Ridge Regression is similar to the Linear Regression, but it adds a regularization term to the cost function, which penalizes the model for having too many features (in this case, the L2 norm of the weights). This is useful to avoid overfitting, and also to perform feature selection, since the regularization term will make the weights of the less important features go to zero.

We can define this regularization term as:

$$
\lambda \sum_{i=1}^{n} w_i^2
$$

Where $\lambda$ is the regularization parameter, and $w_i$ is the weight of the $i$-th feature.

Differently from the Lasso Regression, the Ridge Regression will not make the weights of the less important features go to zero, but it will make them very small (since $w_i$ is squared, it will be even smaller than in the Lasso Regression).

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge()

params = {
    'alpha': np.logspace(-4, 0, 100),
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'max_iter': [15000, 20000, 30000],
    'tol': [0.0001, 0.001],
}

grid = GridSearchCV(ridge, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
ridge_best = Ridge(**best_params)
ridge_best.fit(Xtrain, ytrain)

In [ ]:
scores_ridge = cross_val_score(ridge_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_ridge = np.sqrt(-scores_ridge)
error_percent_ridge = 100 * (10**scores_ridge.mean() - 1)
std_percent_ridge = 100 * (10**scores_ridge.std() - 1)
print(f'Average error is {error_percent_ridge:.2f}%')
print(f'Standard deviation of error is {std_percent_ridge:.2f}%')

path = DATA_DIR / 'processed' / 'ridge.csv'

np.savetxt(path, scores_ridge, delimiter=',')

### Experiment 4:  Decision Tree Regression

Decision trees are a type of algorithm used mostly for classification, since they are easy to interpret and visualize. However, they can also be used for regression. 

From our exploratory analysis, we know that most variables form a linear relationship, however it is still a good itea to test this model because it is a non-parametric model, which means that it does not make any assumptions about the data distribution. This means that it can capture relationships that linear models cannot.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

A decision tree is a tree where each node represents a feature (or a group of features), each link represents a decision (resulting from a feature split), and each leaf represents an output (a prediction). The tree is built by splitting the data into subsets, and then splitting it again on each of the subsets, and so on, until the subsets are small enough to be represented by a leaf.

As for hyperparameters, we can specify:
- min_samples_split: the minimum number of samples required to split an internal node
- min_samples_leaf: the minimum number of samples required to be at a leaf node
- min_weight_fraction_leaf: the minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node
- max_features: the number of features to consider when looking for the best split
- min_impurity_decrease: a node will be split if this split induces a decrease of the impurity greater than or equal to this value
- ccp_alpha: complexity parameter used for Minimal Cost-Complexity Pruning

We will leave max_depth as None, so the nodes will be expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

In [ ]:
tree = DecisionTreeRegressor()

params = {
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3],
    'min_weight_fraction_leaf': [0, 0.01, 0.1],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_impurity_decrease': [0, 0.01, 0.1],
    'ccp_alpha': [0, 0.01, 0.1]
}

grid = GridSearchCV(tree, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
tree_best = DecisionTreeRegressor(**best_params)
tree_best.fit(Xtrain, ytrain)

In [ ]:
scores_tree = cross_val_score(tree_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_tree = np.sqrt(-scores_tree)
error_percent_tree = 100 * (10**scores_tree.mean() - 1)
std_percent_tree = 100 * (10**scores_tree.std() - 1)
print(f'Average error is {error_percent_tree:.2f}%')
print(f'Standard deviation of error is {std_percent_tree:.2f}%')

path = 'tree_scores.csv'

np.savetxt(path, scores_tree, delimiter=',')

### Experiment 5:  Random Forest Regression

The random forest is very similar to the decision tree, but it uses a technique called bagging to reduce the variance of the model, by training many decision trees on different subsets of the data, and then averaging the predictions.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()

params = {
    'n_estimators': [100, 600, 1200, 1800],
    'min_samples_split': [3],
    'max_features': ['auto', 'sqrt'],
    'verbose': [0, 1, 2, 3],
    'max_depth': [None],
    'bootstrap': [False],
    'min_weight_fraction_leaf': [0.0],
    'min_samples_leaf': [1],
}

grid = GridSearchCV(rf, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
rf_best = RandomForestRegressor(**best_params)
rf_best.fit(Xtrain, ytrain)

In [ ]:
scores_rf = cross_val_score(rf_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_rf = np.sqrt(-scores_rf)
error_percent_rf = 100 * (10**scores_rf.mean() - 1)
std_percent_rf = 100 * (10**scores_rf.std() - 1)
print(f'Average error is {error_percent_rf:.2f}%')
print(f'Standard deviation of error is {std_percent_rf:.2f}%')

path = DATA_DIR / 'processed' / 'rf_scores.csv'

np.savetxt(path, scores_rf, delimiter=',')

### Experiment 6:  Gradient Boosting Regression

Gradient boosting is a technique that combines weak learners (in this case, decision trees) to create a strong learner. It is similar to the random forest, but instead of training each tree independently, it trains each tree on the residual of the previous tree.

We can define the gradient boosting algorithm as:

$$
F_0(x) = \underset{\gamma}{\arg\min} \sum_{i=1}^{n} L(y_i, \gamma)
$$

$$
F_m(x) = F_{m-1}(x) + \underset{\gamma}{\arg\min} \sum_{i=1}^{n} L(y_i, F_{m-1}(x_i) + \gamma h_m(x_i))
$$

Where $F_0(x)$ is the first tree (obtained by minimizing $L(y_i, \gamma)$ over the training data), $F_m(x)$ is the $m$-th tree, $L$ is the loss function, $y_i$ is the target value, $x_i$ is the $i$-th feature vector, and $h_m(x_i)$ is the prediction of the $m$-th tree.

The technique uses the gradient descent algorithm to minimize the loss function, and the trees are added sequentially, so the model is built in a stage-wise fashion.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor()

params = {
 'alpha': [0.9], 
 'criterion': ['friedman_mse'],
 'learning_rate': [0.1, 0.5, 0.01], 
 'max_depth': [3, 4, 5], 
 'max_features': ['sqrt'], 
 'min_samples_leaf': [2,3], 
 'n_estimators': [1600,200], 
 'subsample': [0.9, 0.8, 0.7, 0.6], 
 'verbose': [0,1,2], 
 'warm_start': [True, False]
}

grid = GridSearchCV(gbr, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
with open('best_params_gradient.txt', 'w') as f:
    f.write(str(best_params))
print(best_params)

In [ ]:
gbr_best = GradientBoostingRegressor(**best_params)
gbr_best.fit(Xtrain, ytrain)

In [ ]:
scores_gbr = cross_val_score(gbr_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_gbr = np.sqrt(-scores_gbr)
error_percent_gbr = 100 * (10**scores_gbr.mean() - 1)
std_percent_gbr = 100 * (10**scores_gbr.std() - 1)
print(f'Average error is {error_percent_gbr:.2f}%')
print(f'Standard deviation of error is {std_percent_gbr:.2f}%')

path = DATA_DIR / 'processed' / 'gradient_scores.csv'

np.savetxt(path, scores_gbr, delimiter=',')

### Experiment 7:  KNN Regression

KNN is a different non-parametric approach to regression. It is a lazy learning algorithm, which means that it does not learn a function from the training data, but instead memorizes the training data (that is, stores everything on memory) and waits until it is given a new data point to make a prediction.

During training, the algorithm simply stores the training data. During testing, the algorithm finds the $k$ nearest neighbors of the new data point, and then averages the target values of the neighbors to make a regression prediction.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()

params = {
    'n_neighbors': [5, 6, 7, 8],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2, 3],
    'leaf_size': [30, 40, 50]
}

grid = GridSearchCV(knn, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
knn_best = KNeighborsRegressor(**best_params)
knn_best.fit(Xtrain, ytrain)

In [ ]:
scores_knn = cross_val_score(knn_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_knn = np.sqrt(-scores_knn)
error_percent_knn = 100 * (10**scores_knn.mean() - 1)
std_percent_knn = 100 * (10**scores_knn.std() - 1)
print(f'Average error is {error_percent_knn:.2f}%')
print(f'Standard deviation of error is {std_percent_knn:.2f}%')

path = DATA_DIR / 'processed' / 'knn_scores.csv'

np.savetxt(path, scores_knn, delimiter=',')


### Experiment 8: ElasticNetCV

ElasticNet is a combination of Lasso and Ridge regressions, which adds both regularization terms (L1 and L2) to the cost function. This allows the model to learn a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge.

We can define this new cost function as:

$$

\lambda_1 \sum_{i=1}^{n} |w_i| + \lambda_2 \sum_{i=1}^{n} w_i^2

$$

Where $\lambda_1$ and $\lambda_2$ are the regularization parameters.

It is important to note that ElasticNetCV is not a model, but a method that can be used to find the best values for the regularization parameters $\lambda_1$ and $\lambda_2$.

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
elnet = ElasticNetCV()

params = {
    'l1_ratio': [0.6, 0.3, 0.2, 0.1],
    'eps': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8],
    'n_alphas': [100, 150, 200],
    'copy_X': [True, False],
    'verbose': [0,1,2]
}

grid = GridSearchCV(elnet, params, cv=5, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
elnet_best = ElasticNetCV(**best_params)
elnet_best.fit(Xtrain, ytrain)

In [ ]:
scores_elnet = cross_val_score(elnet_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_elnet = np.sqrt(-scores_elnet)
error_percent_elnet = 100 * (10**scores_elnet.mean() - 1)
std_percent_elnet = 100 * (10**scores_elnet.std() - 1)
print(f'Average error is {error_percent_elnet:.2f}%')
print(f'Standard deviation of error is {std_percent_elnet:.2f}%')

path = DATA_DIR / 'processed' / 'elnet_scores.csv'

np.savetxt(path, scores_elnet, delimiter=',')

### Retraining of best model on the whole dataset

In [ ]:
best_model = linear_scaled_model

In [ ]:
best_model.fit(X, y)

## Chossing the best model

In [ ]:
from scipy.stats import ttest_ind

def compare_scores(score1, score2):
    t_stat, p_value = ttest_ind(score1, score2, equal_var=False)
    print(f'T-statistic: {t_stat:.2f}')
    print(f'p-value: {p_value:.2f}')
    if p_value < 0.05:
        print('The difference is statistically significant.')
    else:
        print('Nothing can be said about the difference.')

In [ ]:
path = DATA_DIR / 'processed' / 'models_log.csv'

scores = pd.read_csv(path, index_col=0)

scores.head()

In [ ]:
scores['Accuracy'] = 100 - scores['Mean Absolute Error']
scores.sort_values(by='Accuracy', ascending=False, inplace=True)
scores

In [ ]:
base_line_score = scores.loc['Linear Regression Scaled', 'Accuracy']
base_line_score

In [ ]:
# running the test for all models, related to the baseline

for model in scores.index:
    if model == 'Linear Regression Scaled':
        continue
    print(f'Comparing {model} to baseline...')
    compare_scores(scores.loc[model, 'Accuracy'], base_line_score)
    print()
